In [8]:
from WindPy import w
import pandas as pd
import numpy as np
import datetime as dt
import os

In [9]:
# 本程序用于每天早晨，可更新昨日收盘数据，并基于此进行条款跟踪与策略回测。
w.stop()
w.start()
path = r"C:\Users\Zhwch\Desktop\HaiFuTong\Project\市场数据\转债数据库\转债价格数据库_每日更新" #保存到本地时需修改路径
path = os.getcwd()
os.chdir(path)

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [10]:
df = pd.read_csv(r"转债价格数据库_每日更新.csv",encoding="gbk", low_memory=False) #保存到本地时需修改文件名
df

,日期,代码,转债价格,纯债价值,转换价值,存量,成交额,纯债溢价率,转股溢价率,到期收益率,评级,转债简称,转股价格,正股价格（前复权）,正股涨跌幅,正股成交额,到期日,赎回公告日,主体评级展望
0,2017/1/3,110030.SH,114.040,96.6452,80.5785,9.780730,0.041600,17.998618,41.5266,-1.6792,AA,格力转债(退市),7.26,5.3527,0.006884682,0.253800,2019/12/25,NaN,稳定
1,2017/1/3,110031.SH,108.500,93.7625,47.4801,23.995080,0.082600,15.717904,128.5188,0.6282,AAA,航信转债(退市),43.05,18.4445,0.024561404,2.752400,2021/6/12,NaN,稳定
2,2017/1/3,110032.SH,111.290,91.1285,83.3109,44.998370,0.319500,22.124253,33.5837,-0.0899,AA+,三一转债(退市),7.49,5.5448,0.02295082,1.139900,2022/1/4,2019/3/4,负面
3,2017/1/3,110033.SH,115.320,92.7442,93.9529,27.996640,0.035100,24.342007,22.7423,-0.4489,AA+,国贸转债(退市),8.93,6.1552,0.023170732,1.274200,2022/1/5,2021/10/12,稳定
4,2017/1/3,110034.SH,123.530,91.5688,113.2433,14.996850,0.137000,34.904028,9.0831,-2.0241,AA+,九州转债(退市),18.65,12.7843,0.018322083,1.243200,2022/1/15,NaN,稳定
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416780,2024/02/19,128138.SZ,117.224,117.6819,32.8691,4.198741,0.083969,-0.389100,256.6389,5.2338,AA-,侨银转债,25.13,8.2600,0.0403022670025186,0.396598,2026-11-17,NaN,稳定
416781,2024/02/19,128141.SZ,118.001,104.8648,92.1176,12.701435,0.219078,12.526800,28.0982,-1.4489,AA,旺能转债,14.97,13.7900,-0.01076040172166437,0.427121,2026-12-17,NaN,稳定
416782,2024/02/19,128142.SZ,107.000,104.8568,53.8462,7.179605,0.082079,2.043900,98.7141,2.0739,AA,新乳转债,18.33,9.8700,0.013347022587268942,0.810064,2026-12-18,NaN,稳定
416783,2024/02/19,128143.SZ,130.310,97.7403,106.1758,1.730914,6.004661,33.322700,22.7304,-3.0629,A+,锋龙转债,12.63,13.4100,0.02444614209320095,3.069519,2027-01-08,NaN,稳定


In [11]:
recording_date = dt.datetime.strptime(df["日期"].iloc[-2],"%Y/%m/%d")
print(recording_date)
s_date = str(recording_date + dt.timedelta(1)).split()[0]
#s_date = dt.datetime.strptime("2019-01-01","%Y-%m-%d")
#t_date = "2023-07-10"  #手动输入最新更新的交易日

today = dt.datetime.today().hour
if today >= 16:
    t_date = str(dt.datetime.today()).split()[0]
else:
    t_date = str(dt.datetime.today()- dt.timedelta(days=1)).split()[0]

date_list = []

date_list = w.tdays(s_date, t_date, "").Data[0]
date_list

2024-02-19 00:00:00


[datetime.datetime(2024, 2, 20, 0, 0)]

In [12]:
if date_list==[]:
    raise Exception("已更新至最新数据，无需再次收集。")

df_f = pd.DataFrame()

for day in date_list:
    print(day)
    day_s = dt.datetime.strftime(day,"%Y/%m/%d")
    #导出当日的所有转债代码
    data = w.wset("sectorconstituent","date=%s;sectorid=1000040882000000"% day_s,use_df=True)
    df = pd.DataFrame(data.Data,columns = data.Codes,index=data.Fields ).T
    code = df.wind_code.tolist()
    data = w.wss(code, "ipo_date",use_df=True)
    df = pd.DataFrame(data.Data,columns = data.Codes,index=data.Fields ).T
    code = df[(df["IPO_DATE"]<=day) & (df["IPO_DATE"]>=pd.to_datetime("2017/1/1"))].index.tolist()
    #提取当日转债价格、纯债价值、转换价值、存量、纯债溢价率、转股溢价率、评级数据
    # data = w.wss(code,"close,strbvalue,convvalue,outstandingbalance,amt,strbpremiumratio,convpremiumratio,ytm_cb,rate_ratebond,sec_name,clause_conversion2_swapshareprice, actualmaturitydate","tradeDate=%s;priceAdj=B;cycle=D;ratingAgency=101;unit=1"%day_s)
    data = w.wss(code,"close,strbvalue,convvalue,outstandingbalance,amt,strbpremiumratio,convpremiumratio,ytm_cb,rate_ratebond,sec_name,clause_conversion2_swapshareprice, actualmaturitydate, rate_fwdissuer,clause_calloption_noticedate","tradeDate=%s;priceAdj=B;cycle=D;ratingAgency=101;type=1;unit=1"%day_s)
    df_d = pd.DataFrame(data.Data,columns = data.Codes,index=data.Fields ).T
    df_d['CLAUSE_CALLOPTION_NOTICEDATE'] = df_d['CLAUSE_CALLOPTION_NOTICEDATE'].replace(pd.to_datetime("1899-12-30"), np.nan)
    df_d["AMT"] = df_d["AMT"]/100000000

    zhenggu = w.wss(code, "cb_pq_stockclose,cb_pq_stockpctchg,cb_pq_stockamnt","startDate=%s;endDate=%s;priceAdj=F;unit=1"%(day_s,day_s))
    df_zhenggu = pd.DataFrame(zhenggu.Data,columns = zhenggu.Codes,index=zhenggu.Fields ).T
    df_zhenggu["CB_PQ_STOCKAMNT"] = df_zhenggu["CB_PQ_STOCKAMNT"]/100000000
    df_zhenggu["CB_PQ_STOCKPCTCHG"] = df_zhenggu["CB_PQ_STOCKPCTCHG"]/100


    df_d[['CB_PQ_STOCKCLOSE', 'CB_PQ_STOCKPCTCHG', 'CB_PQ_STOCKAMNT']] = df_zhenggu[['CB_PQ_STOCKCLOSE', 'CB_PQ_STOCKPCTCHG', 'CB_PQ_STOCKAMNT']]

    df_d["日期"] = day_s
    df_d = df_d.reset_index().set_index("日期")
    df_d = df_d.rename(columns={"index":"代码", "CLOSE":"转债价格", "STRBVALUE":"纯债价值", "CONVVALUE":"转换价值", "OUTSTANDINGBALANCE":"存量", "AMT":"成交额", "STRBPREMIUMRATIO":"纯债溢价率", "CONVPREMIUMRATIO":"转股溢价率", "YTM_CB":"到期收益率", "RATE_RATEBOND":"评级", "SEC_NAME":"转债简称", "CLAUSE_CONVERSION2_SWAPSHAREPRICE":"转股价格", "CB_PQ_STOCKCLOSE":"正股价格（前复权）", "ACTUALMATURITYDATE":"到期日", "CB_PQ_STOCKPCTCHG":"正股涨跌幅", "CB_PQ_STOCKAMNT":"正股成交额", "CLAUSE_CALLOPTION_NOTICEDATE":"赎回公告日", "RATE_FWDISSUER":"主体评级展望"})
    # 把到期日调整为date格式
    df_d["到期日"] = df_d["到期日"].dt.date
    df_d["赎回公告日"] = df_d["赎回公告日"].dt.date
    df_f = pd.concat([df_f,df_d])
# df_f.to_csv("转债价格数据库_最新单日数据.csv",encoding="gbk")
print("%s数据已收集！"%str(t_date))

df = pd.read_csv(r"转债价格数据库_每日更新.csv",encoding="gbk", low_memory=False,index_col=0)
df_new = pd.concat([df, df_f])

df_new.to_csv("转债价格数据库_每日更新.csv",encoding="gbk")
print("%s更新已完成！"%str(t_date))

2024-02-20 00:00:00
2024-02-20数据已收集！
2024-02-20更新已完成！


In [13]:
df_new.loc[df_new.index[-1]]

,代码,转债价格,纯债价值,转换价值,存量,成交额,纯债溢价率,转股溢价率,到期收益率,评级,转债简称,转股价格,正股价格（前复权）,正股涨跌幅,正股成交额,到期日,赎回公告日,主体评级展望
日期,,,,,,,,,,,,,,,,,,
2024/02/20,110044.SH,164.65,107.0922,54.5455,0.70666,0.905879,53.746,201.8581,-69.9551,AA,广电转债,6.82,3.72,0.030471,0.673041,2024-06-27,NaT,稳定
2024/02/20,110045.SH,136.122,106.9985,136.0656,12.94644,1.538026,27.2186,0.0415,-44.3777,AA+,海澜转债,6.1,8.30,0.019656,3.954796,2024-07-15,NaT,稳定
2024/02/20,110047.SH,109.3,110.9734,72.9958,22.46118,0.189255,-1.5079,49.7346,4.5178,AA+,山鹰转债,2.37,1.73,0.0,0.415541,2024-11-21,NaT,负面
2024/02/20,110048.SH,161.123,106.9325,159.9641,11.882391,1.604002,50.6773,0.7245,-38.7499,AA+,福能转债,5.57,8.91,0.027682,2.274767,2024-12-09,NaT,稳定
2024/02/20,110052.SH,122.398,112.1517,91.1839,2.64168,0.391615,9.1361,34.232,-5.921,AA+,贵广转债,7.94,7.24,0.00277,1.069643,2025-03-05,NaT,稳定
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024/02/20,128138.SZ,117.379,117.7366,33.1078,4.198741,0.066488,-0.3037,254.5358,5.1876,AA-,侨银转债,25.13,8.32,0.007264,0.257457,2026-11-17,NaT,稳定
2024/02/20,128141.SZ,118.16,104.8595,91.984,12.701435,0.213498,12.6841,28.4571,-1.498,AA,旺能转债,14.97,13.77,-0.00145,0.267965,2026-12-17,NaT,稳定
2024/02/20,128142.SZ,106.401,104.8515,53.3552,7.179605,0.210225,1.4778,99.4201,2.2821,AA,新乳转债,18.33,9.78,-0.009119,0.383852,2026-12-18,NaT,稳定


In [14]:
# data = w.wset("sectorconstituent","date=%s;sectorid=1000040882000000"%"2023-12-25",use_df=True)
# df = pd.DataFrame(data.Data,columns = data.Codes,index=data.Fields ).T
# code = df.wind_code.tolist()
# data = w.wss(code,"close,strbvalue,convvalue,outstandingbalance,amt,strbpremiumratio,convpremiumratio,ytm_cb,rate_ratebond,sec_name,clause_conversion2_swapshareprice, actualmaturitydate, rate_fwdissuer,clause_calloption_noticedate","tradeDate=%s;priceAdj=B;cycle=D;ratingAgency=101;type=1;unit=1"%"2023-12-25")
# df_d = pd.DataFrame(data.Data,columns = data.Codes,index=data.Fields ).T
# df_d